In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from functools import reduce
import holidays
from stargazer.stargazer import Stargazer
myFmt = mdates.DateFormatter('%m-%d')
plt.rcParams["figure.figsize"] = (11, 5)
np.set_printoptions(suppress=True)

In [2]:
# Load and preprocess the data
demand_df = pd.read_csv(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\Estimation\load\forecast_load.csv")
demand_df = demand_df.rename(columns={'Unnamed: 0': 'date', 'Actual Load': 'load'})
demand_df['date'] = pd.to_datetime(demand_df['date'], utc=True) + pd.DateOffset(hours=1)
demand_df = pd.DataFrame(demand_df.set_index('date')['load'].resample('1h').mean())


In [3]:
demand_df['hour'] = demand_df.index.hour.values.astype(np.int64)
demand_df['year'] = demand_df.index.year.values.astype(np.int64)
demand_df['month'] = demand_df.index.month.values.astype(np.int64)
demand_df['day'] = demand_df.index.day.values.astype(np.int64)
demand_df['date_offset'] = (demand_df.index.month.values * 100 + demand_df.index.day.values - 320) % 1300
demand_df['season'] = pd.cut(demand_df['date_offset'], [0,300, 602, 900, 1300], labels=['spring', 'summer', 'autumn', 'winter'])

demand_df

,load,hour,year,month,day,date_offset,season
date,,,,,,,
2015-01-01 00:00:00+00:00,44600.25,0,2015,1,1,1081,winter
2015-01-01 01:00:00+00:00,43454.75,1,2015,1,1,1081,winter
2015-01-01 02:00:00+00:00,41963.25,2,2015,1,1,1081,winter
2015-01-01 03:00:00+00:00,40617.75,3,2015,1,1,1081,winter
2015-01-01 04:00:00+00:00,39936.75,4,2015,1,1,1081,winter
...,...,...,...,...,...,...,...
2021-12-31 19:00:00+00:00,55785.50,19,2021,12,31,911,winter
2021-12-31 20:00:00+00:00,51848.75,20,2021,12,31,911,winter
2021-12-31 21:00:00+00:00,48751.75,21,2021,12,31,911,winter


In [4]:

demand_df['htype'] = pd.cut(demand_df['hour'], [0, 7, 12, 18, 22, 23], labels=['Off_peak', 'Peak', 'Off_peak', 'Peak', 'Opeak'], ordered=False, include_lowest=True)
demand_df['wday'] = demand_df.index.weekday.values
demand_df['weekday'] = pd.cut(demand_df['wday'], [0, 4, 6], labels=['Wday', 'Wend'], ordered=False, include_lowest=True)
demand_df['wh'] = demand_df['weekday'].astype(str) + "_" + demand_df['htype'].astype(str)
demand_df['date_column'] = demand_df.index
day_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
             7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

demand_df['day'] = demand_df.index.weekday.map(day_map)
demand_df['month'] = demand_df.index.month.map(month_map)

# Create dummy variables
season_dummies = pd.get_dummies(demand_df['season'])
htype_dummies = pd.get_dummies(demand_df['htype'], prefix='htype')
weekday_dummies = pd.get_dummies(demand_df['weekday'], prefix='weekday')
year_dummies = pd.get_dummies(demand_df['year'], prefix='year')
wh_dummies = pd.get_dummies(demand_df['wh'])
day_dummies = pd.get_dummies(demand_df['day'])
month_dummies = pd.get_dummies(demand_df['month'])
hour_dummies = pd.get_dummies(demand_df['hour'], prefix='hour')

demand_df = pd.merge(demand_df, season_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, htype_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, weekday_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, year_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, wh_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, day_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, month_dummies, left_index=True, right_index=True)
demand_df = pd.merge(demand_df, hour_dummies, left_index=True, right_index=True)

# Add holiday data
holiday_dates = list(holidays.DE(years=demand_df['year'].unique()).keys())
holiday_hours = pd.date_range(start=min(holiday_dates), end=max(holiday_dates) + pd.DateOffset(days=1), freq='h')
holidays_date = pd.DataFrame(holiday_hours, columns=['date_column'])
holidays_date['holiday'] = holidays_date['date_column'].dt.date.isin(holiday_dates)
holidays_date['date_column'] = pd.to_datetime(holidays_date['date_column'], utc=True)
demand_df = pd.merge(demand_df, holidays_date, how='left')
demand_df['holiday'] = demand_df['holiday'].fillna(0)
holidays_dummies = pd.get_dummies(demand_df['holiday'], prefix='holidays')

# Merge all dummy variables into the original dataframe using pd.merge

demand_df = pd.merge(demand_df, holidays_dummies, left_index=True, right_index=True)
demand_df['time'] = range(len(demand_df))
demand_df['covid'] = demand_df['spring']*demand_df['year_2020']

demand_df

,load,hour,year,month,day,date_offset,season,htype,wday,weekday,...,hour_19,hour_20,hour_21,hour_22,hour_23,holiday,holidays_False,holidays_True,time,covid
0,44600.25,0,2015,January,Thursday,1081,winter,Off_peak,3,Wday,...,False,False,False,False,False,True,False,True,0,False
1,43454.75,1,2015,January,Thursday,1081,winter,Off_peak,3,Wday,...,False,False,False,False,False,True,False,True,1,False
2,41963.25,2,2015,January,Thursday,1081,winter,Off_peak,3,Wday,...,False,False,False,False,False,True,False,True,2,False
3,40617.75,3,2015,January,Thursday,1081,winter,Off_peak,3,Wday,...,False,False,False,False,False,True,False,True,3,False
4,39936.75,4,2015,January,Thursday,1081,winter,Off_peak,3,Wday,...,False,False,False,False,False,True,False,True,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61363,55785.50,19,2021,December,Friday,911,winter,Peak,4,Wday,...,True,False,False,False,False,0,True,False,61363,False
61364,51848.75,20,2021,December,Friday,911,winter,Peak,4,Wday,...,False,True,False,False,False,0,True,False,61364,False
61365,48751.75,21,2021,December,Friday,911,winter,Peak,4,Wday,...,False,False,True,False,False,0,True,False,61365,False
61366,47461.25,22,2021,December,Friday,911,winter,Peak,4,Wday,...,False,False,False,True,False,0,True,False,61366,False


In [6]:
#
filtered_demand_opeak = demand_df[(demand_df['htype'] == 'Off_peak') & (demand_df['year'] == 2021)]
filtered_demand_peak = demand_df[(demand_df['htype'] == 'Peak') & (demand_df['year'] == 2021)]

p_coef = filtered_demand_peak['load'].mean()/filtered_demand_opeak['load'].mean()
print(p_coef)
mean_load_by_month = np.array(filtered_demand_peak.groupby('month')['load'].mean())

T = 25
np.vstack([mean_load_by_month * 1.023**t for t in range(T)]).reshape(len(mean_load_by_month)*T)/1000

1.0994063118847284


array([ 60.55175556,  56.50764516,  64.14027867,  66.23697817,
        65.05537455,  58.0718414 ,  58.35037222,  64.70375986,
        57.55837366,  65.02603148,  61.30344982,  58.41502685,
        61.94444593,  57.807321  ,  65.61550508,  67.76042867,
        66.55164817,  59.40749375,  59.69243078,  66.19194633,
        58.88221625,  66.52163021,  62.71342917,  59.75857247,
        63.36916819,  59.13688938,  67.1246617 ,  69.31891853,
        68.08233607,  60.77386611,  61.06535669,  67.7143611 ,
        60.23650722,  68.0516277 ,  64.15583804,  61.13301964,
        64.82665906,  60.49703784,  68.66852892,  70.91325366,
        69.6482298 ,  62.17166503,  62.4698599 ,  69.2717914 ,
        61.62194689,  69.61681514,  65.63142231,  62.53907909,
        66.31767222,  61.88846971,  70.24790508,  72.54425849,
        71.25013909,  63.60161332,  63.90666667,  70.86504261,
        63.03925167,  71.21800189,  67.14094503,  63.97747791,
        67.84297868,  63.31190451,  71.8636069 ,  74.21

In [ ]:
mean_load_by_month*1.023

In [ ]:
### Ancienne estimation

X = demand_df[['holidays_True', 'winter', 'summer', 'autumn', 'Wday_Opeak', 'Wend_Peak', 'Wend_Off_peak',
     'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'covid']]
X = sm.add_constant(X)
Y = demand_df[['load']]/1000 # GWh

model = sm.OLS(Y,X.astype(float))
results = model.fit()
print(results.summary())

demand_df['fitted']=results.fittedvalues*1000
demand_df['residuals']= demand_df['load']-demand_df['fitted']


print_results = Stargazer([results])
print_results.render_latex()


In [ ]:
#### Nouvelle estimation

# Référence dummies : Vendredi 15 mai 2015 à 8h

X = demand_df[['holidays_True', 'Wday_Opeak', 'Wend_Peak', 'Wend_Off_peak',
     'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'time', 'covid', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday']]

X = sm.add_constant(X)
Y = demand_df[['load']]/1000 # GWh

model = sm.OLS(Y,X.astype(float))
results = model.fit()
print(results.summary())

demand_df['fitted']=results.fittedvalues*1000
demand_df['residuals']= demand_df['load']-demand_df['fitted']


print_results = Stargazer([results])
print_results.render_latex()



In [ ]:
wind = 144
load_smoothed = demand_df['load'].rolling(window=wind)
fit_smoothed = demand_df['fitted'].rolling(window=wind)
load_smoothed = load_smoothed.mean()/1000
fit_smoothed = fit_smoothed.mean()/1000

In [ ]:
print(demand_df['date_column'])

In [ ]:
demand_df['date_column'] = pd.to_datetime(demand_df['date_column'])
demand_df = demand_df.set_index('date_column')


In [ ]:
# Assuming load_smoothed and fit_smoothed are already defined
fig, ax = plt.subplots(figsize=(12, 6))

# Plotting the data with improved styling# Plotting the data with improved styling
ax.plot(demand_df.index, load_smoothed, color='steelblue', linewidth=1, linestyle='-', label='Realised demand')
ax.plot(demand_df.index, fit_smoothed, color='darkred', linewidth=1, linestyle='-', label='Fitted demand')

# Setting axis labels with larger fonts
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('GWh', fontsize=14)

# Setting the title with a larger font
ax.set_title('Load Estimation', fontsize=16)

# Formatting the x-axis with a date formatter
myFmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(myFmt)

# Adding a legend with a larger font and a frame
ax.legend(frameon=True, facecolor="white", loc='upper right', fontsize=12)

# Adding a grid with light color
ax.grid(color='grey', linestyle='--', linewidth=0.5)

# Setting the background color of the figure to white
fig.patch.set_facecolor('white')

# Saving the figure with high resolution
plt.savefig(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\graphs_results\annexes\load_estimation2.pdf", dpi=300)

# Display the plot
plt.show()

In [ ]:
wind = 24

start_date = '2019-01-01 00:00:00'
end_date = '2019-12-31 23:00:00'
selected_data = demand_df[start_date:end_date]
load_smoothed = selected_data['load'].rolling(window=wind)
fit_smoothed = selected_data['fitted'].rolling(window=wind)
load_smoothed = load_smoothed.mean()/1000
fit_smoothed = fit_smoothed.mean()/1000


In [ ]:

# Plotting the data with improved styling
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(selected_data.index, load_smoothed, color='steelblue', linewidth=1, linestyle='-', label='Realised demand')
ax.plot(selected_data.index, fit_smoothed, color='darkred', linewidth=1, linestyle='-', label='Fitted demand')

# Setting axis labels with larger fonts
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('GWh', fontsize=14)

# Setting the title with a larger font
ax.set_title('Load Estimation', fontsize=16)

# Formatting the x-axis with a date formatter
myFmt = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(myFmt)

# Adding a legend with a larger font and a frame
ax.legend(frameon=True, facecolor="white", loc='upper right', fontsize=12)

# Adding a grid with light color
ax.grid(color='grey', linestyle='--', linewidth=0.5)

# Setting the background color of the figure to white
fig.patch.set_facecolor('white')

# Saving the figure with high resolution
plt.savefig(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\graphs_results\annexes\load_estimation2019.pdf", dpi=300)

# Display the plot
plt.show()


In [ ]:
D_year = {}

demand_df['month'] = demand_df.index.month
demand_df['day'] = demand_df.index.day

for i in range(demand_df['year'].nunique()):
    year_aim = demand_df['year'].min()+i
    D_year[year_aim] = demand_df[demand_df['year']==year_aim][['hour', 'day', 'month', 'residuals', 'fitted']]
    D_year[year_aim] = demand_df[demand_df['year']==year_aim][['hour', 'day', 'month', 'residuals', 'fitted']]
    print(D_year[year_aim])
    D_year[year_aim] = D_year[year_aim].rename(columns={'residuals': 'residuals_'+str(year_aim), 'fitted': 'fitted_'+str(year_aim)})

df_list = [v for k,v in D_year.items()]

df = reduce(lambda df1,df2: pd.merge(df1,df2,on=['day', 'month', 'hour']), df_list)

print(df)

In [ ]:
df['year'] = 2000
df = df.set_index(pd.to_datetime(df[['year', 'month', 'day', 'hour']])).drop(columns=['year', 'month', 'day', 'hour'])

df.to_csv(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\Estimation\load\load2.csv")

In [ ]:
s = np.random.randint(2015,2021,10)

In [ ]:
slice = 199

# Slice the data to plot every other point
load_smoothed = demand_df['load'][::slice]
fit_smoothed = demand_df['fitted'][::slice]

# Create the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plotting the data with improved styling
ax.plot(load_smoothed.index, load_smoothed / 1000, color='blue', linewidth=0.5, label='Realised demand')
ax.plot(fit_smoothed.index, fit_smoothed / 1000, color='red', linestyle='dotted', linewidth=1, label='Fitted demand')

# Setting axis labels with larger fonts
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('GWh (every other point)', fontsize=14)

# Setting the title with a larger font
ax.set_title('Load Estimation', fontsize=16)

# Formatting the x-axis with a date formatter
myFmt = mdates.DateFormatter('%m-%d')
ax.xaxis.set_major_formatter(myFmt)

# Adding a legend with a larger font and a frame
ax.legend(frameon=True, facecolor="white", loc='upper right', fontsize=12)

# Adding a grid with light color
ax.grid(color='grey', linestyle='--', linewidth=0.5)

# Setting the background color of the figure to white
fig.patch.set_facecolor('white')

# Saving the figure with high resolution
plt.savefig(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\Estimation\load\demand_20152.png", dpi=300)

# Display the plot
plt.show()

In [ ]:
load_data = pd.read_csv(r"C:\Users\Alicia BASSIERE\OneDrive - GENES\Documents\Paper 01 - DIPU\Estimation\load\load2.csv", index_col=0)
favorable_load = np.array(load_data['fitted_2021']+load_data['residuals_2021'])
favorable_load[:1897] *= 1.12 # 20 March 
favorable_load[:8497] *= 1.12
favorable_shock = load_data['fitted_2021']-favorable_load
load_data["low_shock"] = favorable_shock

bad_load = np.array(load_data['fitted_2021']+load_data['residuals_2021'])
bad_load[:1897] *= 0.88 # 20 March 
bad_load[:8497] *= 0.88
bad_shock = load_data['fitted_2021']-bad_load
load_data["high_shock"] = bad_shock

load_data